# S2W7D1: 模型架构与分类头设计 (Model Architecture & The "Head")

今天是构建“机器人意图识别系统”的第一天。我们不急着训练，先要把**骨架**搭好。

虽然 Hugging Face 提供了 `AutoModelForSequenceClassification`，但作为算法工程师，**手写一个 Classifier Class** 是必须掌握的基本功。只有这样，你才能随心所欲地修改网络结构（比如在 BERT 后面加个 LSTM，或者改用多层 MLP）。

## 1 🛠️ 核心代码：封装 BERT 分类器

请在 `project_root/src/models/` 目录下新建文件 `model_bert.py`。这段代码将定义我们未来一周要训练的模型本体。

**文件路径**: `project_root/src/models/model_bert.py`

```python
import torch
import torch.nn as nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, model_path, num_labels):
        """
        初始化 BERT 分类器
        :param model_path: 本地预训练模型的路径 (str)
        :param num_labels: 分类的类别数 (int)
        """
        super(BertClassifier, self).__init__()
        
        # 1. 加载预训练的 BERT Backbone (脊柱)
        # config.json 和 model.safetensors 必须在 model_path 下
        self.bert = BertModel.from_pretrained(model_path)
        
        # 2. 定义分类头 (Classification Head)
        # BERT base 的 hidden_size 是 768
        # 这里的逻辑是: [Batch, 768] -> [Batch, num_labels]
        self.classifier = nn.Linear(768, num_labels)
        
        # (可选) Dropout 层，防止过拟合
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        """
        前向传播
        """
        # 1. BERT 编码
        # output[0]: last_hidden_state (Batch, Seq, 768)
        # output[1]: pooler_output (Batch, 768) -> 这是 CLS 经过加工后的向量，适合做分类
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooler_output = outputs.pooler_output
        
        # 2. 经过 Dropout
        pooler_output = self.dropout(pooler_output)
        
        # 3. 经过线性层，得到 Logits
        logits = self.classifier(pooler_output)
        
        return logits
```

## 2 🧠 理论解析：从 Encoder 到 Logits

在 Notebook 中，我们需要验证这个模型是否工作正常。这部分不仅是测试，更是为了让你理解数据的流动。

**Notebook 代码块 (S2W7D1\_Model\_Arch.ipynb):**

In [1]:
import sys
import torch
sys.path.append("../../") 
from src.config import PRETRAINED_MODEL_DIR
from src.models.model_bert import BertClassifier
from transformers import BertTokenizer

# 1. 模拟业务场景：机器人意图定义
# 假设我们有 4 类意图
label_map = {
    0: "MOVE (移动)",
    1: "GRAB (抓取)",
    2: "STOP (停止)",
    3: "OTHER (闲聊/无关)"
}
num_labels = len(label_map)

# 2. 实例化模型
print(f"正在加载模型权重: {PRETRAINED_MODEL_DIR} ...")
# 注意：这里我们使用了 src/config.py 定义的路径，非常优雅
model = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=num_labels)
print("✅ 模型实例化成功！")

# 3. 构造一个 Dummy Input (假数据)
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
text = "请你拿起那个杯子"
inputs = tokenizer(text, return_tensors="pt")

# 4. 前向传播测试
model.eval() # 切换到评估模式
with torch.no_grad():
    logits = model(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )

print("-" * 30)
print(f"输入文本: {text}")
print(f"输出 Logits 形状: {logits.shape}") # 预期: [1, 4]
print(f"输出 Logits 值: {logits}")

# 5. 理解 Logits (未归一化的概率)
# 还没经过 Softmax，所以值可以是负无穷到正无穷
# 训练初期，这些值是随机初始化的，没有意义
predicted_id = torch.argmax(logits, dim=-1).item()
print(f"随机初始化预测结果: {label_map[predicted_id]}")

正在加载模型权重: /home/goodminton/study/AI-Interview-Sprint/data/pretrained_models/bert-base-chinese ...
✅ 模型实例化成功！
------------------------------
输入文本: 请你拿起那个杯子
输出 Logits 形状: torch.Size([1, 4])
输出 Logits 值: tensor([[ 0.1656, -0.5437, -0.7109, -0.7762]])
随机初始化预测结果: MOVE (移动)


## 3 🤖 具身智能场景：为什么需要分类头？

你可能会问：*“为什么不直接用 BERT 输出的向量做相似度匹配（像 Day 3 那样），而非要训练一个分类头？”*

这是\*\*“检索式 (Retrieval)”**与**“分类式 (Classification)”\*\*的区别，也是面试常考点。

  * **相似度匹配 (Day 3)**:
      * **优点**: Zero-shot，不需要重新训练，新加指令只需加到数据库。
      * **缺点**: 难以处理复杂逻辑。例如 "不要停" 和 "停" 语义极度相似，余弦相似度极高，但意图完全相反。
  * **分类头微调 (Week 7)**:
      * **优点**: **精度极高**。通过微调，模型会强行记住 "不要 X" 和 "X" 的区别。对于 "紧急停止" 这种高风险指令，必须用分类模型。
      * **具身应用**: 机器人的安全指令（Stop, E-Stop）通常走分类模型，因为容错率低。

## 4 💥 面试必问：Pooler Output vs Last Hidden State

面试官：“你在 `model_bert.py` 里用了 `outputs.pooler_output`，它和 `outputs[0]` (last\_hidden\_state) 有什么区别？”

  * **Last Hidden State**: `[Batch, Seq_Len, 768]`。是每个 Token 的向量。如果你要做**序列标注**（比如找出句子里的物体名词），必须用这个。
  * **Pooler Output**: `[Batch, 768]`。是 `[CLS]` 位置的向量经过 `Dense` + `Tanh` 激活后的结果。它是官方预训练时用于 NSP 任务的，天生适合做**整句分类**。
  * **面试加分项**: *“其实现在的 Paper 里（如 RoBERTa），有时候建议直接取 `Last Hidden State` 的第一个向量（即 CLS 原始向量）自己做 Pooling，因为 BERT 原生的 Pooler 层在未微调前可能带有一些预训练任务的 Bias，但在全量微调下，直接用 Pooler Output 也是标准做法。”*

## 5 ⚔️ 每日算法题 (LeetCode Daily)

为了配合 Tokenizer 的**滑动窗口**思维，今天我们挑战一道经典的“窗口”题。

### 🎯 题目: [LeetCode 3. 无重复字符的最长子串 (Longest Substring Without Repeating Characters)](../../LeetCode%20practice/1-50.ipynb)

  * **难度**: Medium
  * **描述**: 给定一个字符串 `s` ，请你找出其中不含有重复字符的 **最长子串** 的长度。
  * **示例**: 输入 `s = "abcabcbb"`, 输出 `3` (因为 "abc")。
  * **关联**:
      * **NLP**: 这就是最基础的 **Sliding Window (滑动窗口)** 机制。BERT 的 Attention Mask 其实也是一种控制窗口的手段。
      * **思维**: 维护一个 `left` 和 `right` 指针，以及一个 Hash Set（或字典）来记录窗口内的字符。
  * **Key Idea**: 当 `right` 遇到重复字符时，`left` 要一步步右移，直到把那个重复字符“挤出”窗口。

## ✅ Day 1 任务清单

1.  **工程搭建**: 按照新结构创建目录，编写 `src/config.py`。
2.  **核心代码**: 完成 `src/model_bert.py`，确保没有语法错误。
3.  **跑通 Demo**: 在 `S2W7D1` Notebook 中成功加载模型并打印出 Logits 形状 `[1, 4]`。
4.  **算法**: 通过 LeetCode 3。